# ETL - Sucursales - **CHardy Tecno Store**
---
## 1. Extracción.
### 1.1. importamos el DataFrame Sucursaless.csv

In [1]:
import pandas as pd
from pathlib import Path

# Ruta al archivo Parquet
ruta_archivo = Path("../data/raw/sucursales.csv")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    
    df = pd.read_csv(ruta_archivo)

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df)}")
    print(f" Columnas: {len(df.columns)}")


except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

 Extracción correcta
 Registros cargados: 27
 Columnas: 4


## 2. Transformación
### 2.1. Verificamos los Datos Extraidos

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sucursal_id         27 non-null     int64 
 1   nombre_sucursal     27 non-null     object
 2   direccion_sucursal  26 non-null     object
 3   provincia_sucursal  26 non-null     object
dtypes: int64(1), object(3)
memory usage: 996.0+ bytes


In [3]:
df.head(30)

,sucursal_id,nombre_sucursal,direccion_sucursal,provincia_sucursal
0,1,Sucursal 1,"Av. Neuquén N° 226 Torre 4 Dto. 4, Neuquén 830...",Misiones
1,2,Sucursal 2,"Av. Merlo N° 7959, Río Gallegos 9400, Santa Cruz",Chaco
2,3,Sucursal 3,"Calle Saavedra N° 135 Local 6, Ushuaia 9410, T...",Buenos Aires
3,4,Sucursal 4,"Av. Jujuy N° 75, Viedma 8500, Río Negro",Misiones
4,5,Sucursal 5,"Av. 1 N° 731, Rawson 9103, Chubut",Santa Fe
5,6,Sucursal 6,"Av. Santiago del Estero N° 9654, Formosa 3600,...",Chaco
6,7,Sucursal 7,"Diagonal Mitre N° 5276 Dto. 7, Río Gallegos 94...",Cordoba
7,8,Sucursal 8,"Diagonal Neuquén N° 533, Santiago del Estero 4...",Santa Fe
8,9,Sucursal 9,"Calle Rivadavia N° 38 Piso 8 Dto. 6, Rawson 91...",Salta
9,10,Sucursal 10,"Calle 2 N° 8958 Oficina 9, Constitución 1004, ...",Buenos Aires


In [4]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()] # toda la fila debe estar duplicada
print(df_duplicados)

df_duplicados_id = df[df.duplicated(subset=["sucursal_id"], keep=False)]
print("sucursal_id:\n", df_duplicados_id)


Empty DataFrame
Columns: [sucursal_id, nombre_sucursal, direccion_sucursal, provincia_sucursal]
Index: []
sucursal_id:
 Empty DataFrame
Columns: [sucursal_id, nombre_sucursal, direccion_sucursal, provincia_sucursal]
Index: []


In [5]:
# Detectar duplicados por direccion + provincia
df_duplicados = df[df.duplicated(subset=["direccion_sucursal", "provincia_sucursal"], keep=False)]

print(">>> Duplicados detectados (direccion + provincia):")
print(df_duplicados)

# Control: mostrar agrupados por direccion/provincia con conteo
df_check = (
    df.groupby(["direccion_sucursal", "provincia_sucursal"])
      .size()
      .reset_index(name="conteo")
      .query("conteo > 1")
)

print("\n>>> Direcciones/provincias repetidas con conteo:")
print(df_check)


>>> Duplicados detectados (direccion + provincia):
    sucursal_id nombre_sucursal  \
24           25     Sucursal 25   
25           26     Sucursal 26   

                                   direccion_sucursal provincia_sucursal  
24  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones  
25  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones  

>>> Direcciones/provincias repetidas con conteo:
                                   direccion_sucursal provincia_sucursal  \
14  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones   

    conteo  
14       2  


In [6]:
# Marcar duplicados por direccion + provincia
df["es_duplicado_dirprov"] = df.duplicated(
    subset=["direccion_sucursal", "provincia_sucursal"], keep=False
)

# Control: mostrar las filas marcadas como duplicadas
print(">>> Sucursales marcadas como duplicadas (direccion + provincia):")
print(df[df["es_duplicado_dirprov"]])

# Control: resumen de direcciones/provincias repetidas
df_check = (
    df.groupby(["direccion_sucursal", "provincia_sucursal"])
      .size()
      .reset_index(name="conteo")
      .query("conteo > 1")
)
# dejo marcadas las sucursales repetidas, NO elimino repetida, ya que podrian estar siendo usadas por los registros de ventas
print("\n>>> Direcciones/provincias repetidas con conteo:")
print(df_check)


>>> Sucursales marcadas como duplicadas (direccion + provincia):
    sucursal_id nombre_sucursal  \
24           25     Sucursal 25   
25           26     Sucursal 26   

                                   direccion_sucursal provincia_sucursal  \
24  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones   
25  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones   

    es_duplicado_dirprov  
24                  True  
25                  True  

>>> Direcciones/provincias repetidas con conteo:
                                   direccion_sucursal provincia_sucursal  \
14  Calle Jujuy N° 557, San Salvador de Jujuy 4600...           Misiones   

    conteo  
14       2  


In [7]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

sucursal_id             0
nombre_sucursal         0
direccion_sucursal      1
provincia_sucursal      1
es_duplicado_dirprov    0
dtype: int64


In [8]:
# Valores únicos de la columna 
valores_unicos = df["provincia_sucursal"].unique()
print(valores_unicos)

['Misiones' 'Chaco' 'Buenos Aires' 'Santa Fe' 'Cordoba' 'Salta' 'Neuquén'
 'Santa Fe ' 'Bunoes Aires' 'Tucumán' nan]


In [9]:
# Elimina espacios al inicio y al final('Santa Fe ')
df["provincia_sucursal"] = df["provincia_sucursal"].str.strip()

# Reemplazar valores exactos en la columna 'provincia_sucursal'
df["provincia_sucursal"] = df["provincia_sucursal"].replace({
    "Bunoes Aires": "Buenos Aires",
    "Cordoba": "Córdoba"
})

# Eliminar filas donde 'provincia_sucursal' sea NaN
df = df.dropna(subset=["provincia_sucursal"])

In [10]:
# Reviso los Cambios hechos
valores_unicos = df["provincia_sucursal"].unique()
print(valores_unicos)

print(df.isnull().sum()) # Muestra valores nulos, NaN (valor cero no es nulo)

['Misiones' 'Chaco' 'Buenos Aires' 'Santa Fe' 'Córdoba' 'Salta' 'Neuquén'
 'Tucumán']
sucursal_id             0
nombre_sucursal         0
direccion_sucursal      0
provincia_sucursal      0
es_duplicado_dirprov    0
dtype: int64


## 3. Carga.
### 3.1. Guardamos la dimensión en formato parquet.

In [11]:
import os

filename = "../data/processed/dim_sucursales.parquet"

try:
    df.to_parquet(filename, engine="fastparquet")
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: ../data/processed/dim_sucursales.parquet
